In [81]:
import os
import re
import random
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
import dataloader
from MBnet_val import mobilenet_v3_small

import warnings
warnings.filterwarnings('ignore')



In [82]:
def similarity_class2(table,test_emb):
    global_dis=[]
    for class_list in table:
        local_dis=[] 
        for idx in range(len(class_list)):
            local_dis.append(np.linalg.norm(test_emb[-1]-class_list[idx]))
        dis = np.mean(sorted(local_dis)[3:15])
        global_dis.append(dis)
    return (np.array(global_dis)).argsort()[:]

In [83]:
def embed_shoe(img_array, model_path="/home/jsamare01/DML/mobile_checkpoint/466.ckpt"):

#     img_array = encode_standard_array(path)
    tf.reset_default_graph()
    
    with tf.name_scope('input'):
            inputs = tf.placeholder(tf.float32, [None, 224, 224, 1]) ##輸入為四維[Batch_size,height,width,channels] 
    with tf.name_scope('stem'):

        # out_triplet,out_softmax = MobileV3(inputs)
        softmax, triplet, end_points = mobilenet_v3_small(
            inputs, # input_test
            103,
            multiplier=1.0, 
            is_training=False, 
            reuse=None
        ) # model: softmax outcome
        
    saver = tf.train.Saver()

    with tf.Session() as sess:
            
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, model_path) 
        pred = sess.run(
                        [triplet], 
                        feed_dict={inputs:img_array}) #list

        pred = pred[0]    

    return pred

In [84]:
def convert_table(table,label):
    all_table=[[] for _ in range(496)]
    class_list=[]
    for i in range(len(table)):
        for vector in table[i]:
            all_table[label[i]].append(vector)
    return all_table

In [85]:
ground_img , val_img= dataloader.dataloader("/home/troy0425/image496_crop")

In [86]:
table = np.load("496_vec_crop_466.npy")
label = np.load("496_lab_crop_466.npy")

truth_table=convert_table(table,label)

In [87]:
import copy

In [96]:
img = list(np.load("img.npy"))

In [97]:
path=["aaa.jpg"]
for img_path in path:
    print('For {}'.format(img_path))
    input_img = dataloader.loaddata(img_path)
    img.append(input_img)
    test_emb = embed_shoe(img)
    ans = similarity_class2(truth_table,test_emb)
    for index in ans[0:]:
        print(ground_img[index][0])
    print("")

For aaa.jpg
inputs Tensor("input/Placeholder:0", shape=(?, 224, 224, 1), dtype=float32)
input_size [224, 224]
['/home/troy0425/image496_crop/Nike KD 5/1933.jpg', 321]
['/home/troy0425/image496_crop/Nike KD Trey 5 II/2439.jpg', 387]
['/home/troy0425/image496_crop/Nike KD 3/3164.jpg', 382]
['/home/troy0425/image496_crop/Nike Air Force Max CB 2 Hyperfuse/497.jpg', 21]
['/home/troy0425/image496_crop/Nike Air Diamond Fury 96/11414.jpg', 465]
['/home/troy0425/image496_crop/Nike Air Humara/20073.jpg', 244]
['/home/troy0425/image496_crop/Nike Zoom Kobe 5/22550.jpg', 274]
['/home/troy0425/image496_crop/Nike Free 5.0 TR Fit 5/24840.jpg', 300]
['/home/troy0425/image496_crop/Nike Tennis Classic 12/2137.jpg', 26]
['/home/troy0425/image496_crop/Nike SB Bruin/24023.jpg', 292]
['/home/troy0425/image496_crop/Nike Air Moc/28804.jpg', 354]
['/home/troy0425/image496_crop/Nike Air Max LeBron 7/1061.jpg', 389]
['/home/troy0425/image496_crop/Nike Shox TLX/13323.jpg', 158]
['/home/troy0425/image496_crop/Nike 